In [1]:
import json
import duckdb
import pyarrow as pa
import copy
import zipfile
import tempfile
from typing import List
import gzip
import os
import pyarrow.compute as pc

with open('1865.json') as f:
    file_json = json.load(f)

In [2]:
# Create an in-memory database
conn = duckdb.connect(':memory:')
cur = conn.cursor()

In [3]:
def read_one(dir_path:str) -> List[dict]:
    li = os.listdir(dir_path)
    
    tempdir = tempfile.mkdtemp()
    with zipfile.ZipFile(os.path.join(dir_path, '0001001_0002000.zip'), 'r') as zip_ref:
        for filename in zip_ref.namelist():
            if filename.endswith('.json.gz'):
                with zip_ref.open(filename, 'r') as file:
                    
                    contents = gzip.decompress(file.read())
                    contents_str = contents.decode('utf-8')
                    # Load the JSON data into a Python object
                    print(filename)
                    data = json.loads(contents_str)
                    # pprint(data['PC_AssaySubmit'])
                    break # Read only 1
    return data
    #         zip_ref.extractall(tempdir)

In [6]:
results_schema = pa.schema([
    pa.field('tid', pa.int64()),
    pa.field('name', pa.string()),
    pa.field('unit', pa.int64()),
    pa.field('sunit', pa.string())
])

# Convert units
unit_map = {
    1: 'ppt',
    2: 'ppm',
    3: 'ppb',
    4: 'mm',
    5: 'um',
    6: 'nm',
    7: 'pm',
    8: 'fm',
    9: 'mgml',
    10: 'ugml',
    11: 'ngml',
    12: 'pgml',
    13: 'fgml',
    14: 'm',
    15: 'percent',
    16: 'ratio',
    17: 'sec',
    18: 'rsec',
    19: 'min',
    20: 'rmin',
    21: 'day',
    22: 'rday',
    23: 'ml-min-kg',
    24: 'l-kg',
    25: 'hr-ng-ml',
    26: 'cm-sec',
    27: 'mg-kg',
    254: 'none',
    255: 'unspecified'
}

activity_map = {
    1: 'Inactive',
    2: 'Active',
    3: 'Inconclusive'
}

def format_file(file_json:dict) -> pa.lib.Table:
    # Extract from file data
    data_copy = copy.deepcopy(file_json['PC_AssaySubmit']['data'])
    
    ########## Format data_copy ##########
    pylist = []
    for sid_entry in data_copy:
        sid_results = sid_entry.pop('data') # List of dicts

        # Extract useful data for each TID
        for i, tid_data in enumerate(sid_results):
            tid_data.update({str(tid_data.pop('tid')): # use string TID as dict key
                             list(tid_data.pop('value').values())[0]}) # list of one element
            sid_results[i] = tid_data # over-write

        # Convert list of dictionaries to single dict
        sid_results = {k: v for d in sid_results for k, v in d.items()}

        sid_entry.update(sid_results)
        pylist.append(sid_entry)
        
    data_table = pa.Table.from_pylist(pylist) # convert to table
        
    ########## Join data_copy w/ results_table on TIDs ##########
    exclude_names = ['sid', 'version', 'outcome', 'rank'] # non-TID column names
    # List of TID column names in data_table
    old_names = [int(x) for x in data_table.column_names if x not in exclude_names]

    # List of names mapped to TIDs in results_table
    results_table = pa.Table.from_pylist(file_json['PC_AssaySubmit']['assay']['descr']['results'], schema=results_schema)
    
    ########## Add units to name in results_table ##########
    results_dict = results_table.to_pydict()

    for i in range(len(results_table)):
        unit = results_dict['unit'][i]
        sunit = results_dict['sunit'][i]
        name = results_dict['name'][i]

        if unit is not None:
            results_dict['name'][i] = f'{name}, {unit_map[unit]}'
        elif sunit is not None:
            results_dict['name'][i] = f'{name}, {sunit}'

    results_table = pa.Table.from_pydict(results_dict)

    new_names = [x[1] for x in duckdb.sql(f'SELECT * FROM results_table WHERE tid IN {tuple(old_names)}').fetchall()]
    
    data_table = data_table.rename_columns(exclude_names + new_names)
    
    
    ########## Convert 1, 2, 3 coded activity to strings w/ meaning (same format as PubChem) ##########
    query = f'''
    SELECT
      CASE
        WHEN outcome = 1 THEN '{activity_map[1]}'
        WHEN outcome = 2 THEN '{activity_map[2]}'
        WHEN outcome = 3 THEN '{activity_map[3]}'
        ELSE 'unknown'
      END AS Activity
    FROM data_table;
    '''

    activity_col = pa.array(duckdb.sql(query).fetchall()).flatten()
    data_table = data_table.append_column('Activity', activity_col)
    # Remove old integer activity 'outcome' column
    data_table = data_table.remove_column(data_table.column_names.index('outcome'))
    return data_table

In [7]:
file_json = read_one('/Users/collabpharma/Desktop/JSON')
formatted_table = format_file(file_json)

0001001_0002000/1001.json.gz


In [9]:
duckdb.sql('SELECT * FROM formatted_table').df()

,sid,version,rank,EC50_Qualifier,"EC50, um","AF_20uM, none","Mean_NC, CPS","StdDev_NC, CPS","Mean_PC, CPS","StdDev_PC, CPS",Activity
0,7978331,0,41,>,100.000,9.8,9.0922,1.18180,293.55,18.068,Inactive
1,14729515,0,41,>,100.000,3.5,9.2602,0.91976,350.27,24.225,Inactive
2,24803103,0,41,>,100.000,16.7,9.6639,1.21630,309.08,14.972,Inactive
3,17406996,0,46,=,35.584,3.1,10.3760,3.80620,296.85,19.452,Active
4,17511822,0,41,>,100.000,6.3,8.4894,0.90999,307.33,18.124,Inactive
...,...,...,...,...,...,...,...,...,...,...,...
195627,842136,0,0,NaN,NaN,1.0,9.5194,1.34770,322.25,21.664,Inactive
195628,842128,0,0,NaN,NaN,1.0,9.7825,1.25700,299.69,15.531,Inactive
195629,842127,0,0,NaN,NaN,1.0,9.8048,0.84005,310.28,24.166,Inactive
195630,842125,0,0,NaN,NaN,1.0,9.4109,0.96143,317.79,24.508,Inactive


In [ ]:
results_table = pa.Table.from_pylist(file_json['PC_AssaySubmit']['assay']['descr']['results'], schema=results_schema)

results_dict = results_table.to_pydict()

for i in range(len(results_table)):
    unit = results_dict['unit'][i]
    sunit = results_dict['sunit'][i]
    name = results_dict['name'][i]
    
    if unit is not None:
        results_dict['name'][i] = f'{name}, {unit_map[unit]}'
    elif sunit is not None:
        results_dict['name'][i] = f'{name}, {sunit}'

results_table = pa.Table.from_pydict(results_dict)

In [ ]:
# conn.register('formatted_table', formatted_table)

In [ ]:
# Convert 1, 2, 3 coded activity to strings w/ meaning (same format as PubChem)
activity_map = {
    1: 'Inactive',
    2: 'Active',
    3: 'Inconclusive'
}

query = f'''
SELECT
  CASE
    WHEN outcome = 1 THEN '{activity_map[1]}'
    WHEN outcome = 2 THEN '{activity_map[2]}'
    WHEN outcome = 3 THEN '{activity_map[3]}'
    ELSE 'unknown'
  END AS Activity
FROM formatted_table;
'''

activity_col = pa.array(duckdb.sql(query).fetchall()).flatten()
formatted_table = formatted_table.append_column('Activity', activity_col)

In [ ]:
# formatted_table
duckdb.sql('SELECT * FROM formatted_table')

In [ ]:
# Convert units
unit_map = {
    1: 'ppt',
    2: 'ppm',
    3: 'ppb',
    4: 'mm',
    5: 'um',
    6: 'nm',
    7: 'pm',
    8: 'fm',
    9: 'mgml',
    10: 'ugml',
    11: 'ngml',
    12: 'pgml',
    13: 'fgml',
    14: 'm',
    15: 'percent',
    16: 'ratio',
    17: 'sec',
    18: 'rsec',
    19: 'min',
    20: 'rmin',
    21: 'day',
    22: 'rday',
    23: 'ml-min-kg',
    24: 'l-kg',
    25: 'hr-ng-ml',
    26: 'cm-sec',
    27: 'mg-kg',
    254: 'none',
    255: 'unspecified'
}

query = f'''
SELECT
  CASE
'''

duckdb.sql('SELECT * FROM results_table')

#     WHEN outcome = 1 THEN '{activity_map[1]}'
#     WHEN outcome = 2 THEN '{activity_map[2]}'
#     WHEN outcome = 3 THEN '{activity_map[3]}'
#     ELSE 'unknown'
#   END AS Activity
# FROM formatted_table;
# '''


In [ ]:
results_schema = pa.schema([
    pa.field('tid', pa.int64()),
    pa.field('name', pa.string()),
    pa.field('unit', pa.int64()),
    pa.field('sunit', pa.string())
])


results_table = pa.Table.from_pylist(file_json['PC_AssaySubmit']['assay']['descr']['results'], schema=results_schema)